In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb

import feather
import warnings
import time
import sys
import datetime
import re
import gc

from sklearn.model_selection import KFold
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)

from scipy.stats import mode
from scipy import stats
from sklearn import preprocessing


from IPython.core.display import display, HTML

In [2]:
all_transactions = feather.read_dataframe('all_transactions')

In [3]:
all_transactions['non_installments_purchases'] = (all_transactions.installments == 0) * all_transactions.purchase_amount

In [4]:
all_transactions.drop(['cat2_dum_nan'],axis=1,inplace=True)

In [8]:
def get_non_zero(x: pd.core.series.Series) -> pd.core.series.Series:
    """returns a copy of an array with zero elemnts removed"""
    return x.reset_index(drop=True)[x.nonzero()[0]]

def count_unique(x: pd.core.series.Series) -> pd.core.series.Series:
    """returns a reversed array of unique values' counts"""
    return np.unique(x,return_counts=True)[1][::-1]

In [31]:
def modeplus(x): return mode(x)[0]
def non_zero_mean(x): return get_non_zero(x).mean()
def non_zero_percent(x): return x.astype(np.bool_).mean()
def count_non_zeros(x): return x.astype(np.bool_).sum()
def active_days(x): return x.ptp()

def top_quarter_quantile(x): return x.quantile(q=0.15,interpolation='lower')
def bottom_quarter_quantile(x): return x.quantile(q=0.85,interpolation='higher')

def table_ends(x): return np.sum([(x == -15).max(),(x == 0).max()*2])


In [7]:
toadd = feather.read_dataframe('PandIformatted')

In [ ]:
def modeplus(x):
    return mode(x)[0]

In [47]:
def ptpplus(x): return x.ptp()
agg = {
    'authorized_flag':['mean'],
    'city_id':['mean','std',modeplus,'nunique'],
    'category_1':['mean'],
    'installments':['mean','std','sum',non_zero_mean,count_non_zeros,non_zero_percent],
    'merchant_category_id': ['mean','std',modeplus,'nunique'],
    'month_lag': [table_ends,'nunique'],
    'purchase_amount': ['mean','sum',top_quarter_quantile,bottom_quarter_quantile,'min','max','std','count'],
    'state_id':['mean','std',modeplus,'nunique'],
    'subsector_id': ['mean','std',modeplus,'nunique'],
    'no_city':'mean',
    'undefined_installments':'mean',
    'big_installments':'mean',
    'purchase_amount_unusual':'mean',
    'cat2_dum_0.0':'mean', 
    'cat2_dum_1.0':'mean', 
    'cat2_dum_2.0':'mean', 
    'cat2_dum_3.0':'mean',
    'cat2_dum_4.0':'mean',
    'cat2_dum_5.0':'mean',
    'cat3_dum_A':'mean',
    'cat3_dum_B':'mean',
    'cat3_dum_C':'mean',
    'cat3_dum_nan':'mean',
    'purchase_date_day':[lambda x: (x.nunique()/x.ptp()),ptpplus,'std','min','max'], #check min max to find out real lenght
    'purchase_date_weekend':['mean','std'],
    'purchase_date_hour':['mean',bottom_quarter_quantile,top_quarter_quantile,'std'],
    'purchase_date_day_cycle':'mean',
    'payment_per_month':['std'],
    'number_of_purchases':'max',
    'valid_authorization_percent':'max',
    'non_installments_purchases':['mean','sum',top_quarter_quantile,bottom_quarter_quantile,'min','max','std'],   
}

In [51]:
agged_table = all_transactions.groupby(['card_id']).agg(agg)

In [73]:
agged_table.reset_index(inplace=True)

In [77]:
#feather.write_dataframe(agged_table,'cat_and_etb\\before_clean')

In [78]:
agged_table.replace([np.inf,-np.inf],np.nan,inplace=True)

In [79]:
agged_table.fillna(0,inplace=True)

In [113]:
def read_data(input_file):
    df = pd.read_csv(input_file)
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    return df

train = read_data('C:\\kdata\\train.csv')
test = read_data('C:\\kdata\\test.csv')

In [114]:
table_date_purch = all_transactions.groupby('card_id').agg({'purchase_date':['max','min']})

In [115]:
table_date_purch.columns = ['_'.join(x) for x in table_date_purch.columns.values]

In [ ]:
toadd['card_id'] = agged_table['card_id']

In [116]:
train = pd.merge(train,table_date_purch,how='left',on='card_id') 
test = pd.merge(test,table_date_purch,how='left',on='card_id')

In [117]:
for df in [train,test]:
    df['days_to_start'] = (df.purchase_date_min-df.first_active_month).dt.days
    df['days_to_end'] = (df.purchase_date_max-df.first_active_month).dt.days

In [118]:
for df in [train,test]:
    df.drop(['first_active_month','purchase_date_max','purchase_date_min'],axis=1,inplace=True)

In [119]:
train = pd.merge(train,agged_table,how='left',on='card_id') 
test = pd.merge(test,agged_table,how='left',on='card_id')

In [120]:
train = pd.merge(train,toadd,how='left',on='card_id') 
test = pd.merge(test,toadd,how='left',on='card_id')

In [122]:
train

,card_id,feature_1,feature_2,feature_3,target,days_to_start,days_to_end,authorized_flag_mean,city_id_mean,city_id_std,city_id_modeplus,city_id_nunique,category_1_mean,installments_mean,installments_std,installments_sum,installments_non_zero_mean,installments_count_non_zeros,installments_non_zero_percent,merchant_category_id_mean,merchant_category_id_std,merchant_category_id_modeplus,merchant_category_id_nunique,month_lag_table_ends,month_lag_nunique,purchase_amount_mean,purchase_amount_sum,purchase_amount_top_quarter_quantile,purchase_amount_bottom_quarter_quantile,purchase_amount_min,purchase_amount_max,purchase_amount_std,state_id_mean,state_id_std,state_id_modeplus,state_id_nunique,subsector_id_mean,subsector_id_std,subsector_id_modeplus,subsector_id_nunique,no_city_mean,undefined_installments_mean,big_installments_mean,purchase_amount_unusual_mean,cat2_dum_0.0_mean,cat2_dum_1.0_mean,cat2_dum_2.0_mean,cat2_dum_3.0_mean,cat2_dum_4.0_mean,cat2_dum_5.0_mean,cat3_dum_A_mean,cat3_dum_B_mean,cat3_dum_C_mean,cat3_dum_nan_mean,purchase_date_day_<lambda>,purchase_date_day_ptpplus,purchase_date_day_std,purchase_date_day_min,purchase_date_day_max,purchase_date_weekend_mean,purchase_date_weekend_std,purchase_date_hour_mean,purchase_date_hour_bottom_quarter_quantile,purchase_date_hour_top_quarter_quantile,purchase_date_hour_std,purchase_date_day_cycle_mean,payment_per_month_std,number_of_purchases_max,valid_authorization_percent_max,non_installments_purchases_mean,non_installments_purchases_sum,non_installments_purchases_top_quarter_quantile,non_installments_purchases_bottom_quarter_quantile,non_installments_purchases_min,non_installments_purchases_max,non_installments_purchases_std,cat2_0_purchases_sum,cat2_0_purchases_mean,cat2_0_purchases_std,cat2_1_purchases_sum,cat2_1_purchases_mean,cat2_1_purchases_std,cat2_2_purchases_sum,cat2_2_purchases_mean,cat2_2_purchases_std,cat2_3_purchases_sum,cat2_3_purchases_mean,cat2_3_purchases_std,cat2_4_purchases_sum,cat2_4_purchases_mean,cat2_4_purchases_std,cat2_5_purchases_sum,cat2_5_purchases_mean,cat2_5_purchases_std,cat3_A_purchases_sum,cat3_A_purchases_mean,cat3_A_purchases_std,cat3_B_purchases_sum,cat3_B_purchases_mean,cat3_B_purchases_std,cat3_C_purchases_sum,cat3_C_purchases_mean,cat3_C_purchases_std,cat3_n_purchases_sum,cat3_n_purchases_mean,cat3_n_purchases_std,0_month_payments_sum,0_month_payments_mean,0_month_payments_std,1_month_payments_sum,1_month_payments_mean,1_month_payments_std,2_month_payments_sum,2_month_payments_mean,2_month_payments_std,3_month_payments_sum,3_month_payments_mean,3_month_payments_std,4_month_payments_sum,4_month_payments_mean,4_month_payments_std,5_month_payments_sum,5_month_payments_mean,5_month_payments_std,6_month_payments_sum,6_month_payments_mean,6_month_payments_std,7_month_payments_sum,7_month_payments_mean,7_month_payments_std,8_month_payments_sum,8_month_payments_mean,8_month_payments_std,9_month_payments_sum,9_month_payments_mean,9_month_payments_std,10_month_payments_sum,10_month_payments_mean,10_month_payments_std,11_month_payments_sum,11_month_payments_mean,11_month_payments_std,12_month_payments_sum,12_month_payments_mean,12_month_payments_std,13_month_payments_sum,13_month_payments_mean,13_month_payments_std,14_month_payments_sum,14_month_payments_mean,14_month_payments_std,15_month_payments_sum,15_month_payments_mean,15_month_payments_std,cat2_0_installments_sum,cat2_0_installments_mean,cat2_0_installments_std,cat2_0_installments_count_non_zeros,cat2_1_installments_sum,cat2_1_installments_mean,cat2_1_installments_std,cat2_1_installments_count_non_zeros,cat2_2_installments_sum,cat2_2_installments_mean,cat2_2_installments_std,cat2_2_installments_count_non_zeros,cat2_3_installments_sum,cat2_3_installments_mean,cat2_3_installments_std,cat2_3_installments_count_non_zeros,cat2_4_installments_sum,cat2_4_installments_mean,cat2_4_installments_std,cat2_4_installments_count_non_zeros,cat2_5_installments_sum,cat2_5_installments_mean,cat2_5_installments_st

In [123]:
feather.write_dataframe(train,'cat_and_etb\\train_cats_and_etb')
feather.write_dataframe(test,'cat_and_etb\\test_cats_and_etb')